In [4]:
# Add new video entries
################################
import os
import datetime
from moviepy.editor import VideoFileClip
import mysql.connector

# Connect to the database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

# Define the folders
video_folders = [
    # Add the paths of other folders you want to traverse here
    "\\\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\wheat_videos\\drone_video_525_623\\original",
    "\\\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\wheat_videos\\drone_video_720\\original",
    "\\\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\wheat_videos\\drone_video_721_727\\original"
]

for video_folder in video_folders:
    # Verify if the folder path is valid and exists
    if not os.path.exists(video_folder) or not os.path.isdir(video_folder):
        print(f"Skipping invalid folder path: {video_folder}")
        continue  # skip to the next folder if the current one is invalid
    
    # List all .MP4 files in the folder
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.MP4')]
    # Sort the files by name
    video_files.sort()

    for video_file in video_files:
        video_path = os.path.join(video_folder, video_file)
        
        # Check if this video entry already exists in the database
        query_check = f"SELECT * FROM video WHERE filename='{video_file}'"
        cursor.execute(query_check)
        if cursor.fetchone():  # Entry exists, skip this file
            print(f"Entry for {video_file} already exists. Skipping...")
            continue
        
        try:
            # Using moviepy to extract video properties
            with VideoFileClip(video_path) as clip:
                frame_rate = clip.fps
                duration = clip.duration
                frame_number = int(frame_rate * duration)

            # Extract creation date and time from file metadata
            creation_timestamp = os.path.getctime(video_path)
            creation_date = datetime.datetime.fromtimestamp(creation_timestamp).date()
            creation_time = datetime.datetime.fromtimestamp(creation_timestamp).time()

            # Video properties
            video_properties = {
                "filename": video_file,
                "location": video_path,
                "create_date": creation_date,
                "create_time": creation_time,
                "duration": duration,
                "format": "MP4",
                "frame_width": 3840,
                "frame_height": 2160,
                "frame_rate": frame_rate,
                "frame_number": frame_number,
                "color_space": "YUV"
            }

            # SQL query to insert video properties into the database
            insert_query = (
                "INSERT INTO video (filename, location, create_date, create_time, duration, format, frame_width, "
                "frame_height, frame_rate, frame_number, color_space) "
                "VALUES (%(filename)s, %(location)s, %(create_date)s, %(create_time)s, %(duration)s, %(format)s, "
                "%(frame_width)s, %(frame_height)s, %(frame_rate)s, %(frame_number)s, %(color_space)s)"
            )
            cursor.execute(insert_query, video_properties)
            cnx.commit()
            print(f"Added entry for {video_file} to the video table.")
        
        except:
            print(f"Failed to add entry for {video_file} to the video table, skipping")
            continue


# Close database connection
cursor.close()
cnx.close()


Entry for DJI_0113.MP4 already exists. Skipping...
Entry for DJI_0114.MP4 already exists. Skipping...
Entry for DJI_0115.MP4 already exists. Skipping...
Entry for DJI_0116.MP4 already exists. Skipping...
Entry for DJI_0117.MP4 already exists. Skipping...
Entry for DJI_0118.MP4 already exists. Skipping...
Entry for DJI_0119.MP4 already exists. Skipping...
Entry for DJI_0120.MP4 already exists. Skipping...
Entry for DJI_0121.MP4 already exists. Skipping...
Entry for DJI_0122.MP4 already exists. Skipping...
Entry for DJI_0123.MP4 already exists. Skipping...
Entry for DJI_0124.MP4 already exists. Skipping...
Entry for DJI_0125.MP4 already exists. Skipping...
Entry for DJI_0126.MP4 already exists. Skipping...
Entry for DJI_0127.MP4 already exists. Skipping...
Entry for DJI_0128.MP4 already exists. Skipping...
Entry for DJI_0129.MP4 already exists. Skipping...
Entry for DJI_0130.MP4 already exists. Skipping...
Entry for DJI_0131.MP4 already exists. Skipping...
Entry for DJI_0132.MP4 already 

In [2]:
# Remove duplicate video entries
################################

import mysql.connector

# Establish a connection to the database
cnx = mysql.connector.connect(
    user='root', password='dronevideos', 
    host='localhost', database='crop_videos'
)
cursor = cnx.cursor()

try:
    # Identify duplicate rows except for the first occurrence
    cursor.execute("""
        CREATE TEMPORARY TABLE duplicates AS
        SELECT video_id
        FROM video
        WHERE (filename)
        IN (
            SELECT filename
            FROM video
            GROUP BY filename
            HAVING COUNT(*) > 1
        )
        AND video_id NOT IN (
            SELECT MIN(video_id)
            FROM video
            GROUP BY filename
        )
    """)
    
    # Delete the identified duplicate rows
    cursor.execute("""
        DELETE FROM video
        WHERE video_id IN (SELECT video_id FROM duplicates)
    """)
    
    # Committing the changes
    cnx.commit()
    
finally:
    # Closing the cursor and connection
    cursor.close()
    cnx.close()

print("Duplicates removed successfully!")



Duplicates removed successfully!


In [3]:
# Update existing video entries
################################

import os
import datetime
from moviepy.editor import VideoFileClip
import mysql.connector

# Connect to the database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

# Define the folder
video_folder = "\\\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\wheat_videos\\drone_videos\\original"
video_files = [f for f in os.listdir(video_folder) if f.endswith('.MP4')]

for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    
    # Using moviepy to extract video properties
    with VideoFileClip(video_path) as clip:
        frame_rate = clip.fps
        duration = clip.duration
        frame_number = int(frame_rate * duration)

    # Extract creation date and time from file metadata
    creation_timestamp = os.path.getctime(video_path)
    creation_date = datetime.datetime.fromtimestamp(creation_timestamp).date()
    creation_time = datetime.datetime.fromtimestamp(creation_timestamp).time()

    # Update query to set new properties for existing rows matching the filename
    update_query = (
        "UPDATE video SET "
        "location = %(location)s, create_date = %(create_date)s, create_time = %(create_time)s, "
        "duration = %(duration)s, format = %(format)s, frame_width = %(frame_width)s, frame_height = %(frame_height)s, "
        "frame_rate = %(frame_rate)s, frame_number = %(frame_number)s "
        "WHERE filename = %(filename)s"
    )
    
    video_properties = {
        "filename": video_file,
        "location": video_path,
        "create_date": creation_date,
        "create_time": creation_time,
        "duration": duration,
        "format": "MP4",
        "frame_width": 3840,  # Or the actual width, if different
        "frame_height": 2160,  # Or the actual height, if different
        "frame_rate": frame_rate,
        "frame_number": frame_number,
    }

    cursor.execute(update_query, video_properties)
    cnx.commit()
    print(f"Updated entry for {video_file} in the video table.")

# Close database connection
cursor.close()
cnx.close()


Updated entry for DJI_0004.MP4 in the video table.
Updated entry for DJI_0012.MP4 in the video table.
Updated entry for DJI_0013.MP4 in the video table.
Updated entry for DJI_0014.MP4 in the video table.
Updated entry for DJI_0015.MP4 in the video table.
Updated entry for DJI_0016.MP4 in the video table.
Updated entry for DJI_0017.MP4 in the video table.
Updated entry for DJI_0018.MP4 in the video table.
Updated entry for DJI_0019.MP4 in the video table.
Updated entry for DJI_0024.MP4 in the video table.
Updated entry for DJI_0025.MP4 in the video table.
Updated entry for DJI_0026.MP4 in the video table.
Updated entry for DJI_0028.MP4 in the video table.
Updated entry for DJI_0029.MP4 in the video table.
Updated entry for DJI_0030.MP4 in the video table.
Updated entry for DJI_0031.MP4 in the video table.
Updated entry for DJI_0032.MP4 in the video table.
Updated entry for DJI_0033.MP4 in the video table.
Updated entry for DJI_0034.MP4 in the video table.
Updated entry for DJI_0035.MP4 

In [5]:
# Add new processed_videos entries
################################
import os
import datetime
from moviepy.editor import VideoFileClip
import mysql.connector

# Connect to the database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

# Define the folders
video_folders = [
    # Add the paths of other folders you want to traverse here
    "\\\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\wheat_videos\\drone_video_525_623\\rescale_crop_fps30",
    "\\\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\wheat_videos\\drone_video_720\\rescale_crop_fps30",
    "\\\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\wheat_videos\\drone_video_721_727\\rescale_crop_fps30"
]

for video_folder in video_folders:
    # Verify if the folder path is valid and exists
    if not os.path.exists(video_folder) or not os.path.isdir(video_folder):
        print(f"Skipping invalid folder path: {video_folder}")
        continue  # skip to the next folder if the current one is invalid
    
    # List all .MP4 files in the folder
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.MP4')]
    # Sort the files by name
    video_files.sort()

    for video_file in video_files:
        video_path = os.path.join(video_folder, video_file)
        
        # Check if this video entry already exists in the database
        query_check = f"SELECT * FROM processed_videos WHERE filename='{video_file}'"
        cursor.execute(query_check)
        if cursor.fetchone():  # Entry exists, skip this file
            # print(f"Entry for {video_file} already exists. Skipping...")
            continue
        
        try:
            # Extract the base filename from the processed video filename
            base_filename = "_".join(video_file.split("_")[-2:])
            
            # Fetch video_id from video table for the base filename
            video_id_query = f"SELECT video_id FROM video WHERE filename='{base_filename}'"
            cursor.execute(video_id_query)
            video_id_row = cursor.fetchone()
            if not video_id_row:
                print(f"No video entry found for base filename {base_filename}. Skipping...")
                continue
            video_id = video_id_row[0]

            # Using moviepy to extract video properties
            with VideoFileClip(video_path) as clip:
                frame_rate = clip.fps
                duration = clip.duration
                frame_number = int(frame_rate * duration)

            # Extract creation date and time from file metadata
            creation_timestamp = os.path.getctime(video_path)
            creation_date = datetime.datetime.fromtimestamp(creation_timestamp).date()
            creation_time = datetime.datetime.fromtimestamp(creation_timestamp).time()

            # Video properties
            video_properties = {
                "filename": video_file,
                "location": video_path,
                "create_date": creation_date,
                "create_time": creation_time,
                "duration": duration,
                "format": "MP4",
                "frame_width": 1024,
                "frame_height": 1024,
                "frame_rate": frame_rate,
                "frame_number": frame_number,
                "color_space": "YUV",
                "video_id": video_id,
                "processing_type": "rescale_crop_fps30", 
            }

            # SQL query to insert video properties into the database
            insert_query = (
                "INSERT INTO processed_videos (filename, location, create_date, create_time, duration, format, frame_width, "
                "frame_height, frame_rate, color_space, video_id, processing_type) "
                "VALUES (%(filename)s, %(location)s, %(create_date)s, %(create_time)s, %(duration)s, %(format)s, "
                "%(frame_width)s, %(frame_height)s, %(frame_rate)s, %(color_space)s, %(video_id)s, %(processing_type)s)" 
            )
            cursor.execute(insert_query, video_properties)
            cnx.commit()
            print(f"Added entry for {video_file} to the processed_videos table.")
        
        except Exception as e:
            print(f"Failed to add entry for {video_file} to the processed_videos table due to error: {e}")
            continue
            # break


# Close database connection
cursor.close()
cnx.close()

Added entry for rescale_crop_fps30_DJI_0526.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0531.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0551.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0552.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0579.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0629.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0630.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0634.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0658.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0674.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0707.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0744.MP4 to the processed_videos table.
Added entry for rescale_crop_fps30_DJI_0765.MP4 to t

In [2]:
import mysql.connector


# Connect to the database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

# Query to describe the processed_videos table
query_describe = "DESCRIBE wind_data"
cursor.execute(query_describe)

# Fetch all column names
columns = cursor.fetchall()

# Extract only the column names from the results
column_names = [column[0] for column in columns]
print("Columns in video table:", column_names)

# ...


Columns in video table: ['wind_data_id', 'wind_speed_avg', 'wind_direction_avg', 'detailed_wind_change_table', 'video_id']
